In [ ]:
import numpy as np
from datetime import date
import matplotlib
import matplotlib.pyplot as plt
import pandas as pd
import astropy.io.fits as pf
from astropy.time import Time, TimeDelta
from lsst_efd_client import EfdClient

In [ ]:
scratchDir = "/scratch/cslage/starTracker"

In [ ]:
client = EfdClient('summit_efd')

## This generates the tpoint input format
### One camera, one date only

In [ ]:
[camera, num] = ['Narrow', 102]
date = 20230221
year = int(date/10000)
month = int((date - 10000 * year)/100)
day = int((date - 10000 * year - 100 * month))
path = f"/project/GenericCamera/{num}/{year}/{month:02}/{day:02}/"
outfilename = f"Tpoint_Input_{camera}_{date}.dat"
outfile = open(scratchDir + "/" + outfilename, 'w')
outfile.write("!" + outfilename + "\n")
outfile.write("!LSST Main Telescope, 2023 Feb 21 \n")
outfile.write(": ALTAZ\n")
outfile.write(": ROTNR\n")
outfile.write("-30 14 40.2\n")
filename = f"StarTracker_Summary_Boresight_{camera}_{date}.txt"
data = np.loadtxt(scratchDir+"/"+filename, skiprows=1)
for j in range(data.shape[0]):
    [seqNum,ra,dec,el,az,raSolve,decSolve,deltaRA,deltDec,elSolve,azSolve,deltaEl,deltaAz,\
     rot,rms_error] = data[j]
    seqNum = int(seqNum)
    filename = path + f"GC{num}_O_{date}_{seqNum:06}.fits"
    hdulist = pf.open(filename)
    header = hdulist[0].header
    expTime = header['EXPTIME']
    start_time = Time(header["DATE-OBS"], scale='tai')
    #time = Time((Time(header["DATE-BEG"], scale='tai').unix_tai + Time(header["DATE-END"], scale='tai').unix_tai)/2.0, format='unix_tai', scale='tai')
    #if seqNum == 160:
    #    print(time.isot, time.utc.isot)
    #    break

    end_time = start_time + TimeDelta(expTime, format='sec')

    azMTM = await client.select_time_series('lsst.sal.MTMount.azimuth', \
                                    ['actualPosition'],  start_time.utc, end_time.utc)
    elMTM = await client.select_time_series('lsst.sal.MTMount.elevation', \
                                    ['actualPosition'],  start_time.utc, end_time.utc) 
    azMTMount = (azMTM.values[:,0][0] + azMTM.values[:,0][-1]) / 2.0
    azMTMount = azMTMount % 360
    elMTMount = (elMTM.values[:,0][0] + elMTM.values[:,0][-1]) / 2.0
    #outfile.write(f"{azSolve:.6f}\t{elSolve:.6f}\t{azMTMount:.6f}\t{elMTMount:.6f}\t{rot:.6f}\n")
#outfile.write("END\n")
#outfile.close()

## This generates the tpoint input format
### One camera, multiple dates

In [ ]:
[camera, num] = ['Narrow', 102]
outfilename = f"Tpoint_Input_Narrow_20230221_22.dat"
outfile = open(scratchDir + "/" + outfilename, 'w')
outfile.write("!" + outfilename + "\n")
outfile.write("!LSST Main Telescope, 2023 Feb 21_22 \n")
outfile.write(": ALTAZ\n")
outfile.write(": ROTNR\n")
outfile.write("-30 14 40.2\n")

dates = [20230221, 20230222]
for date in dates:
    year = int(date/10000)
    month = int((date - 10000 * year)/100)
    day = int((date - 10000 * year - 100 * month))
    path = f"/project/GenericCamera/{num}/{year}/{month:02}/{day:02}/"
    filename = f"StarTracker_Summary_Boresight_{camera}_{date}.txt"
    data = np.loadtxt(scratchDir+"/"+filename, skiprows=1)
    for j in range(data.shape[0]):
        [seqNum,ra,dec,el,az,raSolve,decSolve,deltaRA,deltDec,elSolve,azSolve,deltaEl,deltaAz,\
         rot,rms_error] = data[j]
        seqNum = int(seqNum)
        filename = path + f"GC{num}_O_{date}_{seqNum:06}.fits"
        hdulist = pf.open(filename)
        header = hdulist[0].header
        expTime = header['EXPTIME']
        start_time = Time(header["DATE-OBS"], scale='tai')
        end_time = start_time + TimeDelta(expTime, format='sec')

        azMTM = await client.select_time_series('lsst.sal.MTMount.azimuth', \
                                        ['actualPosition'],  start_time.utc, end_time.utc)
        elMTM = await client.select_time_series('lsst.sal.MTMount.elevation', \
                                        ['actualPosition'],  start_time.utc, end_time.utc) 
        azMTMount = (azMTM.values[:,0][0] + azMTM.values[:,0][-1]) / 2.0
        azMTMount = azMTMount % 360
        elMTMount = (elMTM.values[:,0][0] + elMTM.values[:,0][-1]) / 2.0
        outfile.write(f"{azSolve:.6f}\t{elSolve:.6f}\t{azMTMount:.6f}\t{elMTMount:.6f}\t{rot:.6f}\n")
outfile.write("END\n")
outfile.close()

## This generates the tpoint input format
### Multiple cameras, multiple dates

In [ ]:
cameras = [['Wide', 101], ['Narrow', 102]]
outfilename = f"Tpoint_Input_Test_Both_20230221_22.dat"
outfile = open(scratchDir + "/" + outfilename, 'w')
outfile.write("!" + outfilename + "\n")
outfile.write("!LSST Main Telescope, 2023 Feb 21_22 \n")
outfile.write(": ALTAZ\n")
outfile.write(": ROTNR\n")
outfile.write("-30 14 40.2\n")

dates = [20230221, 20230222]
for [camera, num] in cameras:
    for date in dates:
        year = int(date/10000)
        month = int((date - 10000 * year)/100)
        day = int((date - 10000 * year - 100 * month))
        path = f"/project/GenericCamera/{num}/{year}/{month:02}/{day:02}/"
        filename = f"StarTracker_Summary_Boresight_{camera}_{date}.txt"
        data = np.loadtxt(scratchDir+"/"+filename, skiprows=1)
        for j in range(data.shape[0]):
            [seqNum,ra,dec,el,az,raSolve,decSolve,deltaRA,deltDec,elSolve,azSolve,deltaEl,deltaAz,\
             rot,rms_error] = data[j]
            seqNum = int(seqNum)
            filename = path + f"GC{num}_O_{date}_{seqNum:06}.fits"
            hdulist = pf.open(filename)
            header = hdulist[0].header
            expTime = header['EXPTIME']
            start_time = Time(header["DATE-OBS"], scale='tai')
            end_time = start_time + TimeDelta(expTime, format='sec')

            azMTM = await client.select_time_series('lsst.sal.MTMount.azimuth', \
                                            ['actualPosition'],  start_time.utc, end_time.utc)
            elMTM = await client.select_time_series('lsst.sal.MTMount.elevation', \
                                            ['actualPosition'],  start_time.utc, end_time.utc) 
            azMTMount = (azMTM.values[:,0][0] + azMTM.values[:,0][-1]) / 2.0
            azMTMount = azMTMount % 360
            elMTMount = (elMTM.values[:,0][0] + elMTM.values[:,0][-1]) / 2.0
            outfile.write(f"{azSolve:.6f}\t{elSolve:.6f}\t{azMTMount:.6f}\t{elMTMount:.6f}\t{rot:.6f}\n")
outfile.write("END\n")
outfile.close()